In [ ]:
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
from IPython.display import HTML
import tqdm
import gtsam
from gtsam.symbol_shorthand import X, P
import sln_letter_fit
from sln_letter_fit import FitParams, OptimizationLoggingParams
import loader, plotting

%load_ext autoreload
%autoreload 1
%aimport sln_letter_fit, loader, plotting, sln_stroke_fit

# Normal 1-stage (for reference)

The fit still hasn't converged after 30 iterations

In [ ]:
strokes = loader.load_segments('D', 1)
sol, history, fitter, _ = sln_letter_fit.fit_trajectory(
    strokes,
    fit_params=FitParams(max_iters=30, initialization_strategy_params=' :D '),
    optimization_logging_params=OptimizationLoggingParams(log_optimization_values=True,
                                                          progress_bar_class=tqdm.tqdm_notebook))

In [ ]:
fig, axes = plt.subplots(1, 4, figsize=(17, 5))
plotting.plot_trajectory(axes[0], strokes, history[0], iteration_number=0)
axes[0].set_title('Initial Guess')
plotting.plot_trajectory(axes[1], strokes, history[15], iteration_number=15)
axes[1].set_title('Optimization result after 15 iters')
plotting.plot_trajectory(axes[2], strokes, sol, iteration_number=len(history) - 1)
axes[2].set_title('Optimization result after 30 iters')
plotting.plot_residuals(axes[3], strokes, history)
plotting.plot_residuals(axes[3], strokes, history, relative=True)
axes[3].set_title('Convergence')
axes[3].legend()
fig.suptitle('1-Stage Fit Results', fontsize=24)
plt.show()

# 2-stage solve
We set the `noise_integration_std` very large during the first stage which is the equivalent of fitting just the velocity.

The reason it's equivalent is because setting `noise_integration_std` very large will allow the integration node points (`X[k]` variables) to move directly to the data points after 1 iteration.  Then, the integration factors will try to match the `X[k]` displacements (which are now the same as the data point displacements).

Finally, the second stage resets `noise_integration_std` to the original value to fit the position.

We see from the results that this appears to have already converged by 30 iterations.

In [ ]:
strokes = loader.load_segments('D', 1)
sol, history, fitter, _ = sln_letter_fit.fit_trajectory(
    strokes,
    fit_params=FitParams(max_iters=15,
                         noise_integration_std=1e2,
                         initialization_strategy_params=' :D '),
    optimization_logging_params=OptimizationLoggingParams(log_optimization_values=True,
                                                          progress_bar_class=tqdm.tqdm_notebook))
param_inits = gtsam.Values()
for parami, param in enumerate(sol['params']):
    param_inits.insert(P(parami), param)
sol2, history2, fitter, _ = sln_letter_fit.fit_trajectory(
    strokes,
    fit_params=FitParams(max_iters=15,
                         noise_integration_std=1e-2,
                         initialization_strategy_params=param_inits),
    optimization_logging_params=OptimizationLoggingParams(log_optimization_values=True,
                                                          progress_bar_class=tqdm.tqdm_notebook))

In [ ]:
fig, axes = plt.subplots(1, 4, figsize=(17, 5))
plotting.plot_trajectory(axes[0], strokes, history[0], iteration_number=0)
axes[0].set_title('Initial Guess')
plotting.plot_trajectory(axes[1], strokes, sol, iteration_number=len(history) - 1)
axes[1].set_title('Optimization result after stage 1')
plotting.plot_trajectory(axes[2], strokes, sol2, iteration_number=len(history) + len(history2) - 2)
axes[2].set_title('Optimization result after stage 2')
plotting.plot_residuals(axes[3], strokes, history + history2[1:])
plotting.plot_residuals(axes[3], strokes, history + history2[1:], relative=True)
axes[3].set_title('Convergence')
axes[3].legend()
fig.suptitle('2-Stage Fit Results', fontsize=24)
plt.show()

In [ ]:
fig, ax = plt.subplots(figsize=(6, 4))
anim = plotting.animate_trajectories(ax, [strokes], [(sol2, history + history2)], is_notebook=True)
plt.close()
HTML(anim.to_jshtml())

### Implement the 2-stage function in `sln_letter_fit` and "test" it here

In [ ]:
sol, history, _, _ = sln_letter_fit.fit_trajectory_2_stage(
    strokes,
    fit_params=FitParams(max_iters=30, initialization_strategy_params=' :D '),
    optimization_logging_params=OptimizationLoggingParams(log_optimization_values=True,
                                                          progress_bar_class=tqdm.tqdm_notebook))
fig, axes = plt.subplots(1, 2)
plotting.plot_trajectory(axes[0], strokes, sol, iteration_number=len(history))
plotting.plot_residuals(axes[1], strokes, history)
plotting.plot_residuals(axes[1], strokes, history, relative=True)
axes[1].legend()
plt.show()


In [ ]:
fig, ax = plt.subplots(figsize=(6, 4))
anim = plotting.animate_trajectories(ax, [strokes], [(sol, history)], is_notebook=True)
plt.close()
HTML(anim.to_jshtml())

In [ ]:
fig, ax = plt.subplots()
_, anim = sln_letter_fit.fit_and_plot_trajectories(ax,
                                                   'A',
                                                   num_strokes=None,
                                                   trajectory_indices=None,
                                                   max_iters=30,
                                                   animate=True)
plt.close()
HTML(anim.to_jshtml())